In [26]:
from sklearn import svm
from sklearn.metrics import accuracy_score
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, ParameterGrid
import pandas as pd

In [27]:
def convert_sample(image):
    image = tf.image.rgb_to_grayscale(image)
    image = tf.image.resize(image,[72,72]).numpy()
    image = image.reshape(1,-1)
    return image

In [28]:
X = np.load('Xtrain.npy')
print(X.shape)
X = np.vstack(list(map(convert_sample,X)))
X = StandardScaler(with_mean=0, with_std=1).fit_transform(X)
print(f'Shape of training data features (observations,features): {X.shape}')

y = np.load('ytrain.npy')
y = y.reshape(-1,)    
print(f'Shape of training data labels (observations,): {y.shape}')

Xtest = np.load('Xtest.npy')
Xtest = np.vstack(list(map(convert_sample,Xtest)))
Xtest = StandardScaler(with_mean=0, with_std=1).fit_transform(Xtest)
print(f'Shape of training data features (observations,features): {Xtest.shape}')

(26214, 96, 96, 3)


c:\ProgramData\anaconda3\envs\amlfall23\lib\site-packages\sklearn\utils\_param_validation.py:591: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(


Shape of training data features (observations,features): (26214, 5184)
Shape of training data labels (observations,): (26214,)
Shape of training data features (observations,features): (1638, 5184)


c:\ProgramData\anaconda3\envs\amlfall23\lib\site-packages\sklearn\utils\_param_validation.py:591: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(


In [29]:
X_train, X_val, y_train, y_val = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)

In [30]:
subset_train = np.random.choice(X_train.shape[0], 5000, replace=False)
subset_val = np.random.choice(X_val.shape[0], 2000, replace=False)

X_train_sub = X_train[subset_train]
y_train_sub = y_train[subset_train]

X_val_sub = X_val[subset_val]
y_val_sub = y_val[subset_val]


In [31]:
kernels = ['linear', 'poly', 'rbf'] 
Cs = [0.01, 0.1, 1.0, 2.0, 4.0, 5.0, 10.0, 100.0] 
Ds = [1,2,3,4,5,6]


results = []

for kernel in kernels:
    if kernel == "linear" or kernel == "rbf":
        for C in Cs:
                print(f"Training SVM with kernel = {kernel} and C = {C}...")
                svm_current = svm.SVC(kernel=kernel, C=C, gamma="scale")
                svm_current.fit(X_train_sub, y_train_sub)
                y_val_hat = svm_current.predict(X_val_sub)
                accuracy = accuracy_score(y_val_hat, y_val_sub)

                results.append([accuracy, kernel, C])
    elif kernel == "poly": 
         for C in Cs: 
              for d in Ds: 
                print(f"Training SVM with kernel = {kernel} degree = {d} and C = {C}...")
                svm_current = svm.SVC(kernel=kernel, C=C, degree=d, gamma="scale")
                svm_current.fit(X_train_sub, y_train_sub)
                y_val_hat = svm_current.predict(X_val_sub)
                accuracy = accuracy_score(y_val_hat, y_val_sub)

                results.append([accuracy, kernel, C, d])
                   


results = pd.DataFrame(results)
results.columns = ['Accuracy', 'Kernel', 'C', "d"]
print(results)

Training SVM with kernel = linear and C = 0.01...
Training SVM with kernel = linear and C = 0.1...
Training SVM with kernel = linear and C = 1.0...
Training SVM with kernel = linear and C = 2.0...
Training SVM with kernel = linear and C = 4.0...
Training SVM with kernel = linear and C = 5.0...
Training SVM with kernel = linear and C = 10.0...
Training SVM with kernel = linear and C = 100.0...
Training SVM with kernel = poly degree = 1 and C = 0.01...
Training SVM with kernel = poly degree = 2 and C = 0.01...
Training SVM with kernel = poly degree = 3 and C = 0.01...
Training SVM with kernel = poly degree = 4 and C = 0.01...
Training SVM with kernel = poly degree = 5 and C = 0.01...
Training SVM with kernel = poly degree = 6 and C = 0.01...
Training SVM with kernel = poly degree = 1 and C = 0.1...
Training SVM with kernel = poly degree = 2 and C = 0.1...
Training SVM with kernel = poly degree = 3 and C = 0.1...
Training SVM with kernel = poly degree = 4 and C = 0.1...
Training SVM with 

In [32]:
results[results['Accuracy'] == results['Accuracy'].max()]

,Accuracy,Kernel,C,d
58,0.7655,rbf,1.0,NaN


In [34]:
svm_best = svm.SVC(kernel='rbf', C = 1.0)

# Use both training and validation data to fit it (np.concatenate "stacks" the array like rbind in R)
svm_best.fit(np.concatenate([X_train, X_val]), np.concatenate([y_train, y_val]))



SVC()

In [35]:
# Predict on test data
ytest_hat = svm_best.predict(Xtest)



In [36]:
ytest_hat= pd.DataFrame({
    'Id': list(range(len(ytest_hat))),
    'Predicted': ytest_hat.reshape(-1,),
})
ytest_hat.to_csv("ytest_hat.csv", index=False)

In [ ]:
print(ytest_hat)

        Id  Predicted
0        0          0
1        1          1
2        2          0
3        3          0
4        4          0
...    ...        ...
1633  1633          0
1634  1634          1
1635  1635          1
1636  1636          0
1637  1637          1

[1638 rows x 2 columns]
